In [342]:
import pandas as pd
import numpy as np
import unidecode

from stop_words import get_stop_words
##from p.frequent_patterns import apriori
from apyori import apriori as apriori2
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

In [2]:
precios = pd.read_pickle('precios.pkl')
productos = pd.read_pickle('productos.pkl')
sucursales = pd.read_pickle('sucursales.pkl')

In [3]:
di = {1:1, 2:1, 3:1, 4:2, 5:2, 6:3, 7:3, 8:4, 9:4, 10:4}

In [4]:
precios['periodo'] = precios['medicion'].replace(di)

In [5]:
#pasar a formato columnar
precios2 = precios.pivot_table(index=['producto','sucursal'], columns = 'medicion', values = 'precio')

In [6]:
#rellenar con los precios aledaños
precios2 = precios2.interpolate(axis =1)

In [7]:
#columnas por periodo y final
precios2['p1'] = precios2[[1,2,3]].mean(axis=1)
precios2['p2'] = precios2[[4,5]].mean(axis=1)
precios2['p3'] = precios2[[6,7]].mean(axis=1)
precios2['p4'] = precios2[[8,9,10]].mean(axis=1)
precios2['mean'] = precios2.mean(axis=1)

In [8]:
#eliminar los que tengan faltantes en las nuevas columnas
precios3 = precios2.dropna(subset = ['p1', 'p2', 'p3', 'p4', 'mean'])

In [9]:
def var(row, c1, c2):
    res = (row[c2] - row[c1]) / row[c2]
    return(res)

In [10]:
#variacion intra periodo
precios3['var1'] = (precios3.p2 - precios3.p1 ) / precios3.p1
precios3['var2'] = (precios3.p3 - precios3.p2 ) / precios3.p1
precios3['var3'] = (precios3.p4 - precios3.p3 ) / precios3.p1
precios3['var_tot'] = (precios3.p4 - precios3.p1 ) / precios3.p1

/Users/bsidoruk/Documents/MAESTRIA/MARTES/TP/tp2/env_tp2/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/bsidoruk/Documents/MAESTRIA/MARTES/TP/tp2/env_tp2/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/bsidoruk/Documents/MAESTRIA/MARTES/TP/tp2/env_tp2/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying 

In [11]:
precios3['var1_cat'] = pd.cut(precios3['var1'], 
                              bins=[-1.1, -0.1, -0.05, -0.005, 0.005, 0.05, 0.1, 1.1],
                             labels = ['df', 'dm', 'dl', 'm', 'al', 'am', 'af'])

precios3['var2_cat'] = pd.cut(precios3['var2'], 
                              bins=[-1.1, -0.1, -0.05, -0.005, 0.005, 0.05, 0.1, 1.1],
                             labels = ['df', 'dm', 'dl', 'm', 'al', 'am', 'af'])

precios3['var3_cat'] = pd.cut(precios3['var3'], 
                              bins=[-1.1, -0.1, -0.05, -0.005, 0.005, 0.05, 0.1, 1.1],
                             labels = ['df', 'dm', 'dl', 'm', 'al', 'am', 'af'])

precios3['var_tot_cat'] = pd.cut(precios3['var_tot'], 
                              bins=[-1.1, -0.1, -0.05, -0.005, 0.005, 0.05, 0.1, 1.1],
                             labels = ['df', 'dm', 'dl', 'm', 'al', 'am', 'af'])

/Users/bsidoruk/Documents/MAESTRIA/MARTES/TP/tp2/env_tp2/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/bsidoruk/Documents/MAESTRIA/MARTES/TP/tp2/env_tp2/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/bsidoruk/Documents/MAESTRIA/MARTES/TP/tp2/env_tp2/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value

In [12]:
precios4 = precios3.reset_index().merge(precios3.groupby('producto')[['p1','p2', 'p3', 'p4']].mean(), 
                         how = 'left',
                         left_on = 'producto',
                         right_on = 'producto')

precios4.set_index(['producto','sucursal'], inplace=True)

In [13]:
precios4['pr1'] = (precios4.p1_x - precios4.p1_y ) / precios4.p1_y
precios4['pr2'] = (precios4.p2_x - precios4.p2_y ) / precios4.p2_y
precios4['pr3'] = (precios4.p3_x - precios4.p3_y ) / precios4.p3_y
precios4['pr4'] = (precios4.p4_x - precios4.p4_y ) / precios4.p4_y
precios4

medicion                     1      2      3      4      5       6      7  \
producto      sucursal                                                      
0000040084107 10-1-214   36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-1-219   36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-1-220   36.00  36.00  36.00  36.00  36.00  36.000  39.75   
              10-1-26    36.00  36.00  36.00  36.00  36.00  39.750  43.50   
              10-1-271   36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-1-3     36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-1-30    36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-1-5     36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-1-6     36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-2-115   36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-2-116   36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-2-118   36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-2-120   36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-2-148   44.50  44.50  44.50  44.50  44.50  44.500  44.50   
              10-2-150     NaN    NaN  47.00  47.00  47.00  47.000  47.00   
              10-2-151   36.00  36.00  36.00  36.00  36.00  39.750  43.50   
              10-2-182     NaN    NaN  47.00  47.00  47.00  47.000  47.00   
              10-2-189   36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-2-195   44.50  44.50  44.50  44.50  44.50  44.500  44.50   
              10-2-198   36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-2-206   46.10  48.40  47.00  47.00  47.00  47.000  47.00   
              10-2-215   36.00  36.00  36.00  36.00  36.00  39.750  43.50   
              10-2-228   48.40  48.40  47.00  47.00  47.00  47.000  47.00   
              10-2-237   36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-2-258   36.00  36.00  36.00  36.00  36.00  39.750  43.50   
              10-2-277   46.10  48.40  47.00  47.00  47.00  47.000  47.00   
              10-2-381   36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-2-385   36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-2-386   36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-2-389   36.00  36.00  36.00  36.00  36.00  36.000  43.50   
...                        ...    ...    ...    ...    ...     ...    ...   
7898422746827 9-2-39     42.10  42.10  42.50  42.50  42.50  43.700  44.90   
              9-2-46     41.75  42.49  42.89  42.89  42.89  44.090  45.29   
              9-2-48     41.75  41.75  42.10  42.10  42.10  43.295  44.49   
              9-2-54     41.35  42.10  42.50  42.50  42.50  43.700  44.90   
              9-2-630    41.75  41.75  42.10  42.10  42.10  42.100  44.49   
              9-2-701    41.35  42.10  42.50  42.50  42.50  42.500  44.90   
              9-2-916    42.49  43.89  42.89  42.89  42.89  42.890  48.25   
              9-2-977    41.75  42.49  42.89  42.89  42.89  46.350  45.29   
              9-3-121    37.65  40.69  39.99  39.99  39.99  41.090  42.19   
              9-3-5201   40.80  42.29  42.29  42.29  42.29  44.620  46.95   
              9-3-5206   41.15  41.15  41.89  41.89  41.89  41.890  41.89   
              9-3-5213   47.00  47.00  47.00  47.00  47.00  47.000  47.00   
              9-3-5231   38.15  38.90  40.35  40.25  40.25  41.850  43.45   
              9-3-5259   39.65  39.65  41.10  41.00  41.00  42.625  44.25   
              9-3-5260   38.85  42.10  43.10  41.00  41.00  45.250  44.25   
              9-3-5263   38.85  39.65  41.10  41.99  41.00  45.250  44.25   
              9-3-5961   41.89  41.89  42.65  42.65  42.65  42.650  48.59   
8480017674449 15-1-1066  47.99  49.99  49.99  49.99  49.99  49.990  49.99   
              15-1-1072  47.99  49.99  49.99  49.99  49.99  49.990  49.99

In [14]:
precios4['pr1_cat'] = pd.cut(precios4['pr1'], 
                              bins=[-1.1, -0.1, -0.05, -0.005, 0.005, 0.05, 0.1, 1.1],
                             labels = ['muc', 'mec', 'lec', 'm', 'leb', 'meb', 'mub'])

precios4['pr2_cat'] = pd.cut(precios4['pr2'], 
                              bins=[-1.1, -0.1, -0.05, -0.005, 0.005, 0.05, 0.1, 1.1],
                             labels = ['muc', 'mec', 'lec', 'm', 'leb', 'meb', 'mub'])

precios4['pr3_cat'] = pd.cut(precios4['pr3'], 
                              bins=[-1.1, -0.1, -0.05, -0.005, 0.005, 0.05, 0.1, 1.1],
                             labels = ['muc', 'mec', 'lec', 'm', 'leb', 'meb', 'mub'])

precios4['pr4_cat'] = pd.cut(precios4['pr4'], 
                              bins=[-1.1, -0.1, -0.05, -0.005, 0.005, 0.05, 0.1, 1.1],
                             labels = ['muc', 'mec', 'lec', 'm', 'leb', 'meb', 'mub'])

#le sume axis=1
precios4['prm_cat'] = pd.cut(precios4[['pr1','pr2','pr3','pr4']].mean(axis=1), 
                              bins=[-1.1, -0.1, -0.05, -0.005, 0.005, 0.05, 0.1, 1.1],
                             labels = ['muc', 'mec', 'lec', 'm', 'leb', 'meb', 'mub'])

In [15]:
precios_final = precios4.reset_index()[['producto','sucursal','pr1_cat', 'pr2_cat', 'pr3_cat', 'pr4_cat',
                                       'prm_cat', 'var1_cat','var2_cat', 'var3_cat', 'var_tot_cat']]


In [16]:
productos1= productos.copy()
productos1.head()

,_id,id,marca,nombre,presentacion
0,5cbc69be7af152186c0cd485,0000075027513,DOVE,Desodorante Original Dove 50 Gr,50.0 gr
1,5cbc69be7af152186c0cd488,0000077940131,ARCOR,Turron Arcor 25 Gr,25.0 gr
2,5cbc69be7af152186c0cd489,0000077940704,REXONA,Desodorante en Crema en Pote Odorono Rexona 60 Gr,60.0 gr
3,5cbc69be7af152186c0cd48a,0000077953063,DANETTE,Postre Dulce de Leche Danette Pack 2 Un 190 Gr,190.0 gr
4,5cbc69be7af152186c0cd487,0000077903518,OPERA,Galletitas Obleas Rellena Opera 92 Gr,92.0 gr


In [322]:
#minuscula
productos1[['nombre', 'marca', 'presentacion']] = productos1[['nombre', 'marca', 'presentacion']].apply(lambda x: x.str.lower())

# remover numberos
productos1[['nombre', 'marca', 'presentacion']] = productos1[['nombre', 'marca', 'presentacion']].apply(lambda x: x.str.replace('\d+', ''))

# remover puntuacion
productos1[['nombre', 'marca', 'presentacion']] = productos1[['nombre', 'marca', 'presentacion']].apply(lambda x: x.str.replace('[^\w\s]',''))

# remover tildes
productos1['nombre'] = productos1['nombre'].apply(unidecode.unidecode)
productos1['marca'] = productos1['marca'].apply(unidecode.unidecode)
productos1['presentacion'] = productos1['presentacion'].apply(unidecode.unidecode)

#sacar espacios
productos1[['marca', 'presentacion']] = productos1[['marca', 'presentacion']].apply(lambda x: x.str.strip())


In [323]:
marcas = productos1.marca.unique()
presentaciones = productos1.presentacion.unique()
stop_words = get_stop_words('spanish')

def remover(string, stop_words):
    words = string.split()
    resultwords = [word for word in words if word not in stop_words]
    result = ' '.join(resultwords)
    return(result)


productos1['nombre'] = productos1['nombre'].apply(lambda x: remover(x, marcas))
productos1['nombre'] = productos1['nombre'].apply(lambda x: remover(x, presentaciones))
productos1['nombre'] = productos1['nombre'].apply(lambda x: remover(x, stop_words))


In [324]:
#Se genera lista con palabras que aparecen más de cinco veces en el dataset
bow = pd.Series(' '.join(productos1.nombre).lower().split()).value_counts()
bow = pd.DataFrame(bow)
bow = bow[bow[0] >= 30].index.to_list()
len(bow)

15

In [325]:
productos1['nombre2'] = productos1['nombre'].apply(lambda x: x.split())

In [326]:
def checker(x, word):
    if word in x:
        return True
    else:
        return False


In [350]:
#Llamo dataset de sucursales con la localidad más precio por m2 del dataset anterior

sucursales2=pd.read_pickle('df_sucursales.pkl')
sucursales2=sucursales2[['id','sucursalTipo','local','av','localidad','PRECIO_M2_BARRIO']]

sucursales2['av'].replace({1:'av',0:'no_av'}, inplace=True)



In [351]:
### Unifico los dataframes

In [352]:
sucursales2.head()

,id,sucursalTipo,local,av,localidad,PRECIO_M2_BARRIO
0,3-1-1507,Autoservicio,DEHEZA,av,SAAVEDRA,2365.061414
1,10-3-675,Autoservicio,CARREFOUR,av,NUÑEZ,2597.472491
2,3-1-1506,Autoservicio,DEHEZA,no_av,SAAVEDRA,2365.061414
3,3-1-29,Autoservicio,DEHEZA,no_av,NUÑEZ,2597.472491
4,10-3-300,Autoservicio,CARREFOUR,no_av,NUÑEZ,2597.472491


In [353]:
#Si quisiera agrupar las localidades en zonas más grandes para entender si hay una variación notoria por zona
#Agrupamos acorde a la distribución de precios vista en el tp1
suc=sucursales2.localidad.unique().tolist()

zona1=['PALERMO','BELGRANO','COLEGIALES','RECOLETA','RETIRO','PUERTO MADERO']
zona2=['SAAVEDRA', 'NUÑEZ', 'VILLA URQUIZA','COLEGIALES','VILLA ORTUZAR','CABALLITO','PARQUE CHACABUCO','MONSERRAT','SAN NICOLAS','ALMAGRO']
zona3=[x for x in suc if x not in zona1 and x not in zona2]

sucursales2['zona']=np.where(sucursales2['localidad'].isin(zona1), 'zona1', 
                             (np.where(sucursales2['localidad'].isin(zona2), 'zona2',
                                       (np.where(sucursales2['localidad'].isin(zona3), 'zona3','None')))))

In [354]:
##Generando variable para identificar precio M2 promedio:
#para hacerlo rapideli lleno los nan con el minimo (es el barrio choto)

#sucursales2['PRECIO_M2_BARRIO'].fillna(sucursales2['PRECIO_M2_BARRIO'].min(),inplace=True)


In [355]:
"""
sucursales2['PRECIO_M2_BARRIO']=sucursales2['PRECIO_M2_BARRIO']/100
sucursales2['PRECIO_M2_BARRIO']=sucursales2['PRECIO_M2_BARRIO'].astype(int)
sucursales2['PRECIO_M2_BARRIO']=sucursales2['PRECIO_M2_BARRIO']*100
sucursales2['PRECIO_M2_BARRIO']=sucursales2['PRECIO_M2_BARRIO'].astype(str)
sucursales2['PRECIO_M2_BARRIO']='PRECIO_M2_'+sucursales2['PRECIO_M2_BARRIO']
"""

"\nsucursales2['PRECIO_M2_BARRIO']=sucursales2['PRECIO_M2_BARRIO']/100\nsucursales2['PRECIO_M2_BARRIO']=sucursales2['PRECIO_M2_BARRIO'].astype(int)\nsucursales2['PRECIO_M2_BARRIO']=sucursales2['PRECIO_M2_BARRIO']*100\nsucursales2['PRECIO_M2_BARRIO']=sucursales2['PRECIO_M2_BARRIO'].astype(str)\nsucursales2['PRECIO_M2_BARRIO']='PRECIO_M2_'+sucursales2['PRECIO_M2_BARRIO']\n"

In [356]:
#Unifico con dataset de precios_final

df_unif=pd.merge(precios_final,sucursales2,left_on='sucursal',right_on='id')
df_unif=pd.merge(df_unif,productos1,left_on='producto',right_on='id')

In [368]:
#Elimino variables que no 
df_unif1=df_unif.drop(['pr1_cat','pr2_cat','pr3_cat','pr4_cat','var1_cat','var2_cat','var3_cat','_id','id_x','id_y','nombre', 'sucursal','presentacion','producto','PRECIO_M2_BARRIO','localidad'], axis = 1)


In [369]:
lista=df_unif1.iloc[:len(df_unif1), :15].astype(str).values.tolist()
te = TransactionEncoder()
te_ary = te.fit(lista).transform(lista)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [370]:
for w in bow:
    df[w] = df_unif.nombre2.apply(lambda x: checker(x, w))

In [371]:
print('Número de features: '+ str(len(df.columns.tolist())))

Número de features: 1183


In [376]:
df_apriori.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Supermercado),(av),0.724528,0.585351,0.441855,0.609853,1.041858,0.017752,1.062802
1,(av),(Supermercado),0.585351,0.724528,0.441855,0.754855,1.041858,0.017752,1.123713
2,(af),(Supermercado),0.436431,0.724528,0.316861,0.726028,1.002071,0.000655,1.005478
3,(zona3),(av),0.409738,0.585351,0.291534,0.711512,1.215531,0.051693,1.437319
4,(no_av),(Supermercado),0.414649,0.724528,0.282673,0.681715,0.940909,-0.017752,0.865489


#### Apriori sin min suppport para entender distribucion

In [424]:
frequent_itemsets=apriori(df,min_support=0.01, use_colnames=True).sort_values('support',ascending=False)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.reset_index(inplace=True)

In [425]:
df_apriori=association_rules(frequent_itemsets, metric="confidence", min_threshold=0.2)

#### Cómo se distribuye el soporte

In [421]:
"""
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
fig = plt.figure(figsize=(15,6))
ax = plt.gca()
df_plot=df_apriori[['itemsets','support']]
df_plot['support']=df_plot['support'].round(2)
df_plot=df_plot[['itemsets','support']].groupby('support',as_index=False).agg({'itemsets':'count'})
labels=df_plot['support']
ax.set_xticklabels(labels,rotation=30)
start, end = ax.get_xlim()
ax.xaxis.set_ticks(np.arange(start, end, 3))
df_plot.plot.bar(x='support', y='itemsets', rot=0, ax=ax)
plt.show()
"""

"\nimport matplotlib.pyplot as plt\nimport numpy as np\nimport pandas as pd\nfig = plt.figure(figsize=(15,6))\nax = plt.gca()\ndf_plot=df_apriori[['itemsets','support']]\ndf_plot['support']=df_plot['support'].round(2)\ndf_plot=df_plot[['itemsets','support']].groupby('support',as_index=False).agg({'itemsets':'count'})\nlabels=df_plot['support']\nax.set_xticklabels(labels,rotation=30)\nstart, end = ax.get_xlim()\nax.xaxis.set_ticks(np.arange(start, end, 3))\ndf_plot.plot.bar(x='support', y='itemsets', rot=0, ax=ax)\nplt.show()\n"

In [426]:
df_apriori.sort_values('support',ascending=False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Supermercado),(av),0.724528,0.585351,0.441855,0.609853,1.041858,0.017752,1.062802
1,(av),(Supermercado),0.585351,0.724528,0.441855,0.754855,1.041858,0.017752,1.123713
2,(Supermercado),(af),0.724528,0.436431,0.316861,0.437335,1.002071,0.000655,1.001607
3,(af),(Supermercado),0.436431,0.724528,0.316861,0.726028,1.002071,0.000655,1.005478
4,(zona3),(av),0.409738,0.585351,0.291534,0.711512,1.215531,0.051693,1.437319


In [427]:
df_apriori['antecedents_list']=df_apriori['antecedents'].apply(lambda x: [a for a in x])
df_apriori['consequents_list']=df_apriori['consequents'].apply(lambda x: [a for a in x])

In [428]:
df_apriori['COTO']=df_apriori['antecedents_list'].apply(lambda x: True if "COTO" in x else False)
df_apriori['CARREFOUR']=df_apriori['antecedents_list'].apply(lambda x: True if "CARREFOUR" in x else False)
#df_apriori[(df_apriori['COTO']==True)|(df_apriori['CARREFOUR']==True)]

In [ ]:
1. voy a chequear que supermercados tienen mas confidence para cada zona, luego mostrarlo en una mapa para comparar
2. voy a chequear que supermercados tienen mas confidence para aumentoleve o bajo para entender cual es el que menos aumento
3. voy a chequear que supermercados con producto vino tienen mas confidence para aumentoleve o bajo para entender cual es el que menos aumento

El ultimo debería de dar carrefour. 

In [437]:
zonas=['zona3']
supers=['JUMBO','COTO','CARREFOUR','DISCO','VEA','DIA']

df_apriori['ZONAS']=df_apriori['antecedents_list'].apply(lambda x: 
                                                                True 
                                                                if len(x)==1
                                                                and any(a in x for a in zonas)
                                                               
                                                                else False)
df_apriori['SUPERS']=df_apriori['consequents_list'].apply(lambda x: 
                                                                True 
                                                                if len(x)==1
                                                                and any(a in x for a in supers)
                                                                else False)

df_apriori[(df_apriori['SUPERS']==True)&(df_apriori['ZONAS']==True)].sort_values('confidence',ascending=False)

#ZONA3
#DIA: ZONA3

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_list,consequents_list,COTO,CARREFOUR,SUPER_+_ZONA,VARIACIONES,SUPERS,ZONAS
30,(zona3),(COTO),0.409738,0.379633,0.199961,0.488022,1.285510,0.044411,1.211707,[zona3],[COTO],False,False,False,False,True,True
353,(zona3),(CARREFOUR),0.409738,0.267686,0.083007,0.202586,0.756803,-0.026674,0.918360,[zona3],[CARREFOUR],False,False,False,False,True,True


In [431]:
zonas=['zona1','zona2','zona3']
supers=['JUMBO','COTO','CARREFOUR','DISCO','VEA','DIA']
variaciones=['df', 'dm', 'dl', 'm', 'al', 'am', 'af']

df_apriori['SUPER_+_ZONA']=df_apriori['antecedents_list'].apply(lambda x: 
                                                                True 
                                                                if len(x)==2  
                                                                and any(a in x for a in zonas)
                                                                and any(b in x for b in supers)
                                                                else False)
df_apriori['VARIACIONES']=df_apriori['consequents_list'].apply(lambda x: 
                                                                True 
                                                                if len(x)==1
                                                                and any(a in x for a in variaciones)
                                                                else False)

df_apriori[(df_apriori['SUPER_+_ZONA']==True)&(df_apriori['VARIACIONES']==True)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_list,consequents_list,COTO,CARREFOUR,SUPER_+_ZONA,VARIACIONES
963,"(CARREFOUR, zona1)",(af),0.100397,0.436431,0.044022,0.438481,1.004699,0.000206,1.003652,"[CARREFOUR, zona1]",[af],False,True,True,True
1168,"(DISCO, zona1)",(af),0.095003,0.436431,0.039287,0.413530,0.947527,-0.002176,0.960951,"[DISCO, zona1]",[af],False,False,True,True
1446,"(COTO, zona1)",(af),0.077239,0.436431,0.034189,0.442637,1.014220,0.000479,1.011135,"[COTO, zona1]",[af],True,False,True,True


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_list,consequents_list,vino,COTO,CARREFOUR,SUPER_+_ZONA,VARIACIONES
1564,"(Supermercado, lec, COTO, zona2)",(af),0.023183,0.436431,0.01161,0.500782,1.147448,0.001492,1.128903,"[Supermercado, lec, COTO, zona2]",[af],False,True,False,True,True


In [410]:
df_apriori['SUPER_+_ZONA']

0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
1938    False
1939    False
1940    False
1941    False
1942    False
1943    False
1944    False
1945    False
1946    False
1947    False
1948    False
1949    False
1950    False
1951    False
1952    False
1953    False
1954    False
1955    False
1956    False
1957    False
1958    False
1959    False
1960    False
1961    False
1962    False
1963    False
1964    False
1965    False
1966    False
1967    False
Name: SUPER_+_ZONA, Length: 1968, dtype: bool